# Filling With Packmol

While the MDMC `universe.fill` method is serviceable for simpler systems, it isn't particularly sophisticated. For more complex filled universes, MDMC has an integration with packmol, a package for creating atomic configurations such as bilayers or mixtures. If you are not aware of how packmol works, then please read the user guide [here](https://m3g.github.io/packmol/userguide.shtml).

Note to run this tutorial, you need to have Packmol installed; this is already installed in the Docker container. The MDMC interface to packmol is contained in the objects `MDMC.MD.packmol.PackmolSetup` and `MDMC.MD.packmol.PackmolFiller`.

In [1]:
from MDMC.MD.packmol import PackmolSetup, PackmolFiller

## Creating mixtures

The first example shown is a simple system of water mixed in with ethanol. In a 1:1 molecular ratio.

Firstly, we create the molecules needed for the system (See the notebook on [creating atomic configurations](./creating-atomic-configurations.ipynb) for information on this.)

(NB: It is highly recommend to add any atom labels, dihedrals, bond angles, etc. to the `Molecule`s at this stage to ensure they are present in the final universe, rather than having to go in and add them all manually later.)

In [2]:
from MDMC.MD import Atom, Molecule, Bond, BondAngle
H1 = Atom('H')
H2 = Atom('H', position=[0., 1.63298, 0.])
O = Atom('O', position=[0., 0.81649, 0.57736])
h2o_bonds = Bond((H1, O), (H2, O))
HOH_angle = BondAngle(H1, O, H2)
water_molecule = Molecule(atoms=[H1, H2, O], interactions=[HOH_angle, h2o_bonds], name="water")

h_1 = Atom("H", position=[-1.9237, 0.3850, 0.0000])
h_2 = Atom("H", position=[2.0985, 0.2306, 0.0000])
h_3 = Atom("H", position=[1.1184, -1.0093, 0.8869])
h_4 = Atom("H", position=[1.1184, -1.0093, -0.8869])
h_5 = Atom("H", position=[-0.0227, 1.1812, 0.8852])
h_6 = Atom("H", position=[-0.0227, 1.1812, -0.8852])
c_1 = Atom("C", position=[1.1879, -0.3829, 0.0000])
c_2 = Atom("C", position=[0.0000, 0.5526, 0.0000])
o_1 = Atom("O", position=[-1.1867, -0.2472, 0.0000])
ch_bond = Bond((h_2, c_1), (h_3, c_1), (h_4, c_1), (h_5, c_2), (h_6, c_2))
co_bond = Bond((c_2, o_1))
cc_bond = Bond((c_1, c_2))
oh_bond = Bond((o_1, h_1))
oh_bond_angle = BondAngle(h_1, o_1, c_2)
ethanol_molecule = Molecule(atoms=[h_1,h_2,h_3,h_4,h_5,h_6,c_1,c_2,o_1], interactions=[ch_bond, co_bond, cc_bond, oh_bond, oh_bond_angle], name="ethanol")

Next, put this into a `PackmolSetup` object.

Currently, only single fixed molecules, or boxes, cubes and spheres of molecules are supported.

Here we add a cube of molecules with the `setup.add_cube` method. We provide a desired density
in $Ang^-3$; alternatively one could provide a parameter `n_structures` for the desired number
of `Molecule`s one would like in the space. Later we will see `setup.add_box` for a non-cubic
box; furthermore we have `setup.add_sphere` to add a sphere of given radius.

In [3]:
setup = PackmolSetup()
# Two identically overlapping cubes will create a mixture of water and ethanol in a 1:1 ratio of molecules
setup.add_cube(structure=water_molecule, size=40., density=0.01)
setup.add_cube(structure=ethanol_molecule, size=40., density=0.01)

Next step is to pass this `PackmolSetup` object to a `PackmolFiller` object.

Calling `fill_with_packmol` will setup & run packmol in the background and return a filled universe with the molecules specified.

In [4]:
filler = PackmolFiller(setup_data=setup)
universe = filler.fill_with_packmol()


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.14.2 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Types of coordinate files specified: pdb
  Seed for random number generator:      1234567
  Output file: /home/runner/work/MDMCv0.2_pilot/MDMCv0.2_pilot/doc/how-to/use-MDMC/notebooks/packmol_files/output-universe.pdb
  Reading coordinate file: water-0.pdb
  Reading coordinate file: ethanol-1.pdb
  Number of independent structures:            2
  The structures are: 
  Structure            1 :water-0.pdb(           3  atoms)
  Structure            2 :ethanol-1.pdb(         

*******************************|
          |******************************************

************************|

  Function value from last GENCAN loop: f = .29973E+02
  Best function value before: f = .36415E+05
  Improvement from best function value:    99.92 %
  Improvement from last loop:    99.92 %
  Maximum violation of target distance:     1.412086
  Maximum violation of the constraints: .33922E+01
  All-type function value: .21320E+05

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

  Starting GENCAN loop:            1
  Scaling radii by:    1.1000000000000001     

  Packing:|0                                                             100%|
          |*********************

*********************************************|


          |****************************

...

*********************

*********************

************************|


          |**************

*******

  Function value from last GENCAN loop: f = .14634E-01
  Best function value before: f = .33188E-01
  Improvement from best function value:    55.91 %
  Improvement from last loop:    55.91 %
  Maximum violation of target distance:     0.000000
  Maximum violation of the constraints: .97297E-02

################################################################################

                                 Success! 
              Final objective function value: .14634E-01
              Maximum violation of target distance:   0.000000
              Maximum violation of the constraints: .97297E-02

--------------------------------------------------------------------------------

              Please cite this work if Packmol was useful: 

           L. Martinez, R. Andrade, E. G. Birgin, J. M. Martinez, 
         PACKMOL: A package for building initial configurations for
                   molecular dynamics simulations. 
          Journal of Computational Chemistry, 30:2157-2

Now we can use `view` to look at the solved system. This may be slow - switch the viewer to ASE if it is not loading.

In [5]:
from MDMC.gui import view

view(universe)

## Creating bilayers

`PackmolSetup` boxes, cubes, and spheres can be given a centre, for objects not centred on the origin.

Here we will demonstrate how this can be used to create an interface between benzene and water. We get our benzene molecule from file.

In [6]:
from MDMC.readers.configurations import read

setup_2 = PackmolSetup()
benzene_atoms = read("data/benzene.pdb")
benzene_molecule = Molecule(atoms=benzene_atoms, name='benzene')

We define the centre of our boxes through the `origin` parameter;

In [7]:
setup_2.add_box(structure=water_molecule, lengths=(20., 10., 20.,), origin=(0., 10., 0.), density=0.01)
setup_2.add_box(structure=benzene_molecule, lengths=(20., 10., 20.,), origin=(0., 20., 0.), density=0.01)

and as before, we fill with the `PackmolFiller`.

In [8]:
filler_2 = PackmolFiller(setup_data=setup_2)
universe_2 = filler_2.fill_with_packmol()


################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.14.2 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Types of coordinate files specified: pdb
  Seed for random number generator:      1234567
  Output file: /home/runner/work/MDMCv0.2_pilot/MDMCv0.2_pilot/doc/how-to/use-MDMC/notebooks/packmol_files/output-universe.pdb
  Reading coordinate file: water-0.pdb
  Reading coordinate file: benzene-1.pdb
  Number of independent structures:            2
  The structures are: 
  Structure            1 :water-0.pdb(           3  atoms)
  Structure            2 :benzene-1.pdb(         

In [9]:
view(universe_2)